In [ ]:
from pathlib import Path
import geopandas as gpd
import tomli
from pprint import pprint

from hydromt.data_catalog import DataCatalog

from flood_adapt.api import startup
from flood_adapt.api import events

from utils.fa_scenario_utils import init_scenario, create_event, create_projection, create_strategy, create_scenario

In [ ]:
db_path = Path("/mnt/c/Repositories/DT-flood/FloodAdapt_database")
db_name = "Humber"

In [ ]:
scen_path = db_path / db_name / "test_scenario_toplevel.toml"

with open(scen_path, 'rb') as f:
    scen_dict = tomli.load(f)

In [ ]:
scen_dict

In [ ]:
db = startup.read_database(database_path=db_path, site_name=db_name)

In [ ]:
vars(db)

In [ ]:
db.get_events()

In [ ]:
pprint(vars(db.site.attrs))

In [ ]:
if not db.site.attrs.sfincs.offshore_model:
    print("No offshore")

In [ ]:
dc = DataCatalog(scen_dict["event"]["data_catalogues"])

In [ ]:
if scen_dict["event"]["sfincs_forcing"].values() in dc.sources:
    print("found datasource")

In [ ]:
scen_dict["event"]["sfincs_forcing"]["meteo"] in dc.keys

In [ ]:
scen_dict["event"]["sfincs_forcing"].values()

In [ ]:
dc.keys

In [ ]:
extra_key = False

test_dict = {'a': 1}
if extra_key:
    test_dict['b'] = 2

test_dict

In [ ]:
db_init, scen_init = init_scenario(db_path / db_name, "test_scenario_toplevel.toml")

In [ ]:
new_event = create_event(db_init, scen_init)

In [ ]:
pprint(vars(new_event.attrs))

In [ ]:
new_proj = create_projection(db_init, scen_init)

In [ ]:
pprint(vars(new_proj.attrs))

In [ ]:
db_init.get_projections()["name"]

In [ ]:
for measure in scen_init["strategy"]:
    print(measure)
    print(type(measure))

In [ ]:
for measure in scen_init['strategy']:
    print(f"{measure}: {scen_init['strategy'][measure]}")

## Config File creation tests

In [ ]:
db_init, scen_init = init_scenario(db_path / db_name, "test_scenario_toplevel.toml")

In [ ]:
new_event = create_event(db_init, scen_init)

In [ ]:
new_projection = create_projection(db_init, scen_init)

In [ ]:
new_strategy = create_strategy(db_init, scen_init)

In [ ]:
new_scenario = create_scenario(db_init, scen_init)

## Sketch writing timeseries tides

In [ ]:
from hydromt.data_catalog import DataCatalog
from hydromt_sfincs import SfincsModel

In [ ]:
# init scenario
db_init, scen_init = init_scenario(db_path / db_name, "test_scenario_toplevel.toml")

In [ ]:
# init data catalog
dc = DataCatalog(scen_init["event"]["data_catalogues"])

In [ ]:
db_init.site.attrs.sfincs.overland_model

In [ ]:
sf_name = db_init.site.attrs.sfincs.overland_model
sf_path = db_init.static_path/"templates"/sf_name


In [ ]:
sf = SfincsModel(
    root=sf_path,
    mode='r',
    data_libs=scen_init["event"]["data_catalogues"]
)

In [ ]:
vars(sf)

In [ ]:
sf.get_model_time()

In [ ]:
gtsm = dc.get_geodataset("gtsm_reanalysis_hourly",geom=sf.region, variables=['waterlevel'], time_tuple=sf.get_model_time(), buffer=2000)

In [ ]:
df = gtsm.transpose(..., gtsm.vector.index_dim).to_pandas()
gdf_locs = gtsm.vector.to_gdf()

In [ ]:
gdf_locs

In [ ]:
sf.setup_waterlevel_forcing(geodataset='gtsm_reanalysis_hourly', buffer=2000)

In [ ]:
sf.forcing['bzs'].index

In [ ]:
method_list = [func for func in dir(db_init) if callable(getattr(db_init, func))]
method_list

In [ ]:
db_init.get_outputs()